# <center> CREAR ARCHIVOS</center>

In [ ]:
def crear_fecha():
    
    import time
    from datetime import datetime

    # detiene el tiempo para que no todos los archivos generados queden con la misma fecha de timestamp
    time.sleep(0.1)
        
    # Getting the current date and time
    dt = datetime.now()

    # getting the timestamp
    ts = datetime.timestamp(dt)

    return ts

def generar_archivos(todos_los_archivos=True):

    lista_de_archivos = ['info.txt','Dockerfile','Manifest.yaml','imagen.png','main.py','connection.java','calculos.c','addon.php','utils.gif','query.sql','dataset.avro','link.vs']
    
    lista_de_archivos = lista_de_archivos if todos_los_archivos else [lista_de_archivos[0]]
    
    print('lista_de_archivos = {lista_de_archivos}'.format(lista_de_archivos=lista_de_archivos))
    
    for archivo in lista_de_archivos:
        comando = 'echo {crear_archivo} >> {archivo}'.format(crear_archivo=crear_fecha(), archivo=archivo)
        ejecutar_comando_shell(comando)

    return lista_de_archivos

# <center> ELIMINAR ARCHIVOS</center>

In [ ]:
def listar_archivos_eliminables_en_directorio():
    import os
    
    listado_de_archivos = os.listdir(directory)
    
    archivos_eliminables = [archivo for archivo in listado_de_archivos if '.git' not in archivo]
    
    return archivos_eliminables
    
def eliminar_directorio(nombre_repositorio):
    
    import shutil
    import os
    import stat
    from os import path
    
    for root, dirs, files in os.walk("./{nombre_repositorio}".format(nombre_repositorio=nombre_repositorio)):  
        for dir in dirs:
            os.chmod(path.join(root, dir), stat.S_IRWXU)
        for file in files:
            os.chmod(path.join(root, file), stat.S_IRWXU)
    shutil.rmtree("./{nombre_repositorio}".format(nombre_repositorio=nombre_repositorio))

def eliminar_archivos_directorio():
    import os
    
    lista_de_archivos_eliminables = listar_archivos_eliminables_en_directorio()
    
    for archivo in lista_de_archivos_eliminables:
        os.remove(archivo)
        
    return 'Eliminacion Completada con Exito!!! de {lista_de_archivos_eliminables}'.format(lista_de_archivos_eliminables=lista_de_archivos_eliminables)

# <center> CREAR ARCHIVOS, COMMITS Y PUBLICAR RAMAS AL REPOSITORIO </center>

In [ ]:
def validar_si_existe_rama(nombre_rama, local=False):
    
    comando ='git branch -a'
    resultado = ejecutar_comando_shell(comando, response=True)
    
    rama_base = '{nombre_rama}'.format(nombre_rama=nombre_rama) if local else 'origin/{nombre_rama}'.format(nombre_rama=nombre_rama)

    if rama_base in resultado:
        return True
    return False

def elimina_ramas_si_existen_y_crea_la_rama_local(nombre_rama):
    
    # valida si las ramas existen en REMOTO y LOCAL
    rama_remota = validar_si_existe_rama(nombre_rama=nombre_rama)
    rama_local = validar_si_existe_rama(nombre_rama=nombre_rama, local=True)
    
    # verifica si existe la rama la elimina
    if rama_remota:
        # elimina rama REMOTA existente
        comando = 'git push origin -d {nombre_rama}'.format(nombre_rama=nombre_rama)
        ejecutar_comando_git(comando)

     # verifica si existe la rama la elimina
    if rama_local:                                                                                                            
        # elimina la rama
        comando = 'git branch -D {nombre_rama}'.format(nombre_rama=nombre_rama)
        ejecutar_comando_git(comando)
        
    # crea la rama
    comando = 'git checkout -b {nombre_rama}'.format(nombre_rama=nombre_rama)
    ejecutar_comando_shell(comando)
    
    # verifica las ramas actuales
    ejecutar_comando_shell()
    

def crear_commit_por_cada_archivo(todos_los_archivos):
    
    import datetime

    fecha_ahora = datetime.datetime.now()
    
    # crea el listado de archivos
    lista_de_archivos = generar_archivos(todos_los_archivos=todos_los_archivos)
    
    # basado en el listado de archivos, adiciona uno a uno y crea un commit por cada uno
    for archivo in lista_de_archivos:

        comando = 'git add {archivo}'.format(archivo=archivo)
        ejecutar_comando_git(comando)
        
        comando = 'git commit -m "ADICIONADO ARCHIVO {archivo} {fecha_ahora}"'.format(archivo=archivo, fecha_ahora=fecha_ahora)
        ejecutar_comando_git(comando)

    # verifica si todos los archivos estan en los commits creados
    comando = 'git status'
    ejecutar_comando_git(comando)
    
def generar_archivos_y_hacer_commit(nombre_rama, todos_los_archivos = True):
    
    # elimina cambios a la rama remota si los tiene
    comando = 'git restore .'
    ejecutar_comando_git(comando)
    
    # cambiar a la rama remota
    comando = 'git checkout {nombre_rama_remota}'.format(nombre_rama_remota=nombre_rama_remota)
    ejecutar_comando_git(comando)
    
    # valida si las ramas existen en REMOTO y LOCAL y las elimina
    elimina_ramas_si_existen_y_crea_la_rama_local(nombre_rama)
    
    # adiciona archivos de 1 en 1 y genera 1 commit por cada archivo
    crear_commit_por_cada_archivo(todos_los_archivos=todos_los_archivos)

    # mostrar el log de las ramas en forma resumida y con diagrama de uniones como grafo 
    comando = 'git log --graph --oneline'
    ejecutar_comando_git(comando)

    # crea la rama en remoto
    comando = 'git push --set-upstream origin {nombre_rama}'.format(nombre_rama=nombre_rama)
    ejecutar_comando_git(comando)
    
    ejecutar_comando_shell()

# <center> OBTENER LISTA CON LOS NOMBRES DE TODAS LAS RAMAS LOCALES DE UN REPOSITORIO GIT</center>

In [ ]:
def obtener_lista_nombres_ramas_git():

    stdout = subprocess.check_output('git branch'.split())
    out = stdout.decode()
    branches = [b.strip('* ') for b in out.splitlines()]
    return branches

# <center> EJECUTAR COMANDOS SHELL DESDE FUNCION PYTHON</center>

In [ ]:
import subprocess

def ejecutar_comando_shell(comando='git branch', response=False):

    resultado_comando_ejecutado = subprocess.check_output(comando, shell=True, text=True)
    print('############################################# COMANDO A EJECUTAR #############################################')
    print(comando)
    print('####################################### RESULTADO DE COMANDO EJECUTADO #######################################')
    #para evitar el salto de linea usa el parametro end=''
    print(resultado_comando_ejecutado, end='')
    print('##############################################################################################################')
    print()
    if response:
        return resultado_comando_ejecutado

# <center> FUNCION LAMBDA PARA EJECUTAR COMANDOS GIT</center>

In [ ]:
ejecutar_comando_git = lambda comando : ejecutar_comando_shell(comando)

# <center> CONSULTAR LA VERSION DE GIT</center>

In [ ]:
comando = 'git --version'
ejecutar_comando_git(comando)

# <center> CLONAR UN REPOSITORIO GIT</center>

In [ ]:
repositorio_base = 'https://github.com/JorgeCardona'
nombre_del_repositorio = 'comandos_git'

eliminar_directorio(nombre_repositorio=nombre_del_repositorio)

comando = 'git clone {repositorio_base}/{nombre_del_repositorio}.git'.format(repositorio_base=repositorio_base, nombre_del_repositorio=nombre_del_repositorio)
ejecutar_comando_git(comando)

## FETCH
![alternatvie text](fecth.gif)

## PULL
![alternatvie text](pull.gif)

# <center> CAMBIAR A LA CARPETA PARA TRABAJAR CON EL REPOSITORIO</center>

In [ ]:
#importing the os module
import os

#to get the current working directory
directory = os.getcwd()
directory

In [ ]:
os.chdir('{directory}/{repository}'.format(directory=directory, repository=nombre_del_repositorio))
directory = os.getcwd()
directory

# <center> MOSTRAR INFORMACION DEL REPOSITORIO</center>

In [ ]:
comando = 'git remote show origin'
ejecutar_comando_git(comando)

# <center> LISTAR LAS RAMAS DE UN REPOSITORIO </center>

### LISTA TODAS LAS RAMAS LOCALES Y LA RAMA PRINCIPAL DEL REPOSITORIO

In [ ]:
comando = 'git branch'
ejecutar_comando_git(comando)

### LISTA TODAS LAS RAMAS REMOTAS DEL REPOSITORIO

In [ ]:
comando = 'git branch -r'
ejecutar_comando_git(comando)

### LISTA TODAS LAS RAMAS LOCALES INCLUYENDO LAS RAMAS OCULTAS

In [ ]:
comando = 'git branch -a'
ejecutar_comando_git(comando)

# <center> CREAR UNA RAMA BASADA EN LA RAMA ACTUAL DE UN REPOSITORIO</center>

In [ ]:
comando = 'git branch nueva_rama'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

# <center> CAMBIAR A OTRA RAMA DEL REPOSITORIO</center>

In [ ]:
comando = 'git checkout nueva_rama'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

# <center> CREAR UNA RAMA BASADA EN LA ACTUAL Y CAMBIA A LA RAMA CREADA AL MISMO TIEMPO</center>

In [ ]:
comando = 'git checkout -b rama_acceso_rapido'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

# <center> CREAR NUEVA RAMA LOCAL DE UNA RAMA REMOTA</center>

In [ ]:
nombre_rama_remota = 'multi_commits'
copia_rama_remota = '{nombre_rama_remota}_copia'.format(nombre_rama_remota=nombre_rama_remota)
comando = 'git checkout -b {copia_rama_remota} origin/{nombre_rama_remota}'.format(copia_rama_remota=copia_rama_remota, nombre_rama_remota=nombre_rama_remota)
ejecutar_comando_git(comando)
ejecutar_comando_shell()

# <center> DESCARGAR RAMA REMOTA A LOCAL Y CAMBIAR A LA RAMA BAJADA AL MISMO TIEMPO</center>

In [ ]:
comando = 'git checkout -t origin/{nombre_rama_remota}'.format(nombre_rama_remota=nombre_rama_remota)
ejecutar_comando_git(comando)
ejecutar_comando_shell()

# <center> RENOMBRAR UNA RAMA DEL REPOSITORIO</center>

In [ ]:
comando = 'git checkout {copia_rama_remota}'.format(copia_rama_remota=copia_rama_remota)
ejecutar_comando_git(comando)

comando = 'git branch -m nueva_rama nueva_rama_renombrada'
ejecutar_comando_git(comando)

# Alternativa
comando = 'git branch --move rama_acceso_rapido rama_acceso_rapido_renombrada'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

# <center> COMPARAR LA DIFERENCIA EN CONTENIDO ENTRE 2 RAMAS COMMITs DE UN REPOSITORIO</center>

In [ ]:
# confirmar al repositorio remoto el reset
comando = 'git diff main multi_commits'
ejecutar_comando_git(comando)

# <center> RESTAURAR O RECUPERAR ARCHIVOS MODIFICADOS O ELIMINADOS DE UN REPOSITORIO GIT <br> SOLO MODIFICACIONES EN REPOSITORIO LOCAL, BASADO EN EL ULTIMO COMMIT <br/></center>

In [ ]:
nombre_rama_remota = 'main'
generar_archivos()

In [ ]:
eliminar_archivos_directorio()

### RESTAURAR UN ARCHIVO ESPECIFICO

In [ ]:
comando = 'git restore LICENSE'
ejecutar_comando_git(comando)

### RESTAURAR TODOS LOS ARCHIVOS AL MISMO TIEMPO

In [ ]:
comando = 'git restore .'
ejecutar_comando_git(comando)

# <center> VERIFICAR EL ESTADO DE LOS ARCHIVOS EN UN REPOSITORIO GIT</center>

In [ ]:
generar_archivos()

In [ ]:
comando = 'git status'
ejecutar_comando_git(comando)

# <center> AGREGAR CAMBIOS DE ARCHIVOS A UN REPOSITORIO GIT</center>

### AGREGAR EL CAMBIO DE UN ARCHIVO O UN SOLO ARCHIVO NUEVO

In [ ]:
comando = 'git add info.txt'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

### AGREGAR LOS CAMBIOS DE TODOS LOS ARCHIVOS O TODOS LOS CAMBIOS Y ARCHIVOS NUEVOS

In [ ]:
comando = 'git add .'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

# <center> HACER COMMIT A UN REPOSITORIO GIT</center>

In [ ]:
# solo en este paso, al hacer el primer commit se crea la rama
comando = 'git commit -m "adicionados archivo de prueba"'
ejecutar_comando_git(comando)
        
comando = 'git status'
ejecutar_comando_git(comando)

# <center> PUBLICAR UNA RAMA LOCAL QUE NO EXISTE EN REMOTO A UN REPOSITORIO GIT <br>https://github.com/JorgeCardona/comandos_git</br> </center>


In [ ]:
comando = 'git push --set-upstream origin {copia_rama_remota}'.format(copia_rama_remota=copia_rama_remota)
ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)

# <center> ADICIONAR ARCHIVOS NUEVOS A LA RAMA</center>

In [ ]:
generar_archivos()

# SUBIR NUEVOS ARCHIVOS A UNA RAMA DEL REPOSITORIO  <br>https://github.com/JorgeCardona/comandos_git</br> 

In [ ]:
comando = 'git add .'
ejecutar_comando_git(comando)

comando = 'git commit -m "adicionados archivos nuevos"'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

In [ ]:
# siempre origin para repositorios remotos
comando = 'git push origin'
ejecutar_comando_git(comando)

# <center> ELIMINAR UNA RAMA DE UN REPOSITORIO GIT</center>

## ELIMINAR RAMA DE UN REPOSITORIO LOCAL

In [ ]:
# no se puede eliminar una rama si se esta trabajando en ese monento en ella
comando = 'git checkout main'
ejecutar_comando_git(comando)

ramas_a_eliminar = ['nueva_rama_renombrada','rama_acceso_rapido_renombrada',copia_rama_remota]

# eliminar lista de ramas
for rama in ramas_a_eliminar:

    comando = 'git branch -D {rama}'.format(rama=rama)
    ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)

## ELIMINAR RAMA DE UN REPOSITORIO REMOTO <br>https://github.com/JorgeCardona/comandos_git</br> 

In [ ]:
comando = 'git push origin -d {copia_rama_remota}'.format(copia_rama_remota=copia_rama_remota)
ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)

# <center> LISTAR EL HISTORIAL DE COMMITS DE UN REPOSITORIO GIT</center>

### VER EL HISTORIAL COMPLETO DE LA RAMA MAIN

In [ ]:
comando = 'git log'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

### VER HISTORIAL OTRA RAMA REMOTA

In [ ]:
nombre_rama = 'commit_branch'
generar_archivos_y_hacer_commit(nombre_rama=nombre_rama)

In [ ]:
comando = 'git log'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

### VER HISTORIAL DE LOS ULTIMOS n COMMITS

In [ ]:
comando = 'git log -3' 
ejecutar_comando_git(comando)

ejecutar_comando_shell()

### VER HISTORIAL RESUMIDO DE UNA RAMA

In [ ]:
comando = 'git log --oneline'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

### VER HISTORIAL RESUMIDO DE UNA RAMA DE n COMMITS

In [ ]:
comando = 'git log --oneline -3'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

## VER HISTORIAL RESUMIDO DE UNA RAMA SIN ESTAR EN LA RAMA

In [ ]:
comando = 'git log --oneline main'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

## VER HISTORIAL DE n COMMITS EN UN DIAGRAMA DE GRAFO

In [ ]:
comando = 'git log --graph --oneline'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

### VER HISTORIAL DE n COMMITS Y MOSTRAR LOS CAMBIOS ESPECIFICOS DE ESOS COMMITS

In [ ]:
comando = 'git log -2 -p'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

### VER HISTORIAL DE n COMMITS Y MOSTRAR LOS CAMBIOS ESPECIFICOS DE ESOS COMMITS USANDO **SHOW**

In [ ]:
comando = 'git show -2'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

### VER HISTORIAL RESUMIDO DE n COMMITS Y MOSTRAR LOS CAMBIOS ESPECIFICOS DE ESOS COMMITS

In [ ]:
comando = 'git log --oneline -2 -p'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

# <center> LISTAR EL HISTORIAL DE TODAS LAS ACCIONES REALIZADAS EN UN REPOSITORIO GIT <br> LISTADO DESDE EL MAS NUEVO HASTA EL PRIMER CAMBIO</br></center>

In [ ]:
comando = 'git reflog'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

#### ELIMINAR LA RAMA EN EL REPOSITORIO REMOTO Y LOCAL

In [ ]:
comando = 'git checkout main'
ejecutar_comando_git(comando)

comando = 'git push origin -d {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

comando = 'git branch -D {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)

# <center> MODIFICAR EL MENSAJE DEL ULTIMO COMMIT</center>

In [ ]:
nombre_rama = 'amend_branch'
generar_archivos_y_hacer_commit(nombre_rama=nombre_rama, todos_los_archivos=False)

# EN LA CONSOLA DE VSCODE
# git commit --amend

## SE ABRE AUTOMATICAMENTE EL EDITOR DE TEXTO, reemplazar el emnsaje del commit anterior por el que se quiere y cerrar el archivo
## validar en local con git log, no afecta la hora del commit original con la fecha del mensaje actualizado
## subir los cambios al repositorio¶
# git push --force origin amend_branch

In [ ]:
comando = 'git log'
ejecutar_comando_git(comando)

#### ELIMINAR LA RAMA EN EL REPOSITORIO REMOTO Y LOCAL

In [ ]:
comando = 'git checkout main'
ejecutar_comando_git(comando)

comando = 'git push origin -d {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

comando = 'git branch -D {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)

# <center> REGRESAR A UNA VERSION ANTERIOR DE UN REPOSITORIO GIT</center>

### DETACHED - MOVERSE A UN COMMIT ESPECIFICO Y PONER ESTE COMMIT CON LA NUEVA HEAD

In [ ]:
comando = 'git checkout multi_commits'
ejecutar_comando_git(comando)

comando = 'git checkout d15fad27cb8d1bcb6a0e697ef02e2d787dc51b48'
ejecutar_comando_git(comando)

comando = 'git log --oneline'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

### RETORNA AL HEAD ORIGINAL DE LA RAMA

In [ ]:
comando = 'git switch -'
ejecutar_comando_git(comando)

comando = 'git log --oneline'
ejecutar_comando_git(comando)

ejecutar_comando_shell()

# <center> RESET COMMITs DE UN REPOSITORIO</center>

In [ ]:
nombre_rama = 'reset_branch'
generar_archivos_y_hacer_commit(nombre_rama=nombre_rama)

### SOFT RESET n COMMITs **HEAD~n** MANTENIENDO LOS CAMBIOS QUE SE HABIAN HECHO EN LOS ARCHIVOS LOCALES
![alternatvie text](reset_soft.gif)

In [ ]:
# MANITIENE todos los archivos o cambios hechos despues de este commit Y PUEDEN SER GUARDADOS DE NUEVO
comando = 'git reset --soft HEAD~3'
ejecutar_comando_git(comando)

comando = 'git log'
ejecutar_comando_git(comando)

### HARD RESET n COMMITs **HEAD~n** ELIMINANDO LOS ARCHIVOS Y CAMBIOS QUE SE HABIAN HECHO HASTA ESTE COMMIT
![alternatvie text](reset_hard.gif)
![alternatvie text](reflog_reset.gif)

In [ ]:
# elimina todos los archivos o cambios hechos despues de este commit YA NO SE PUEDEN RECUPERAR ESOS ARCHIVOS
comando = 'git reset --hard HEAD~7'
ejecutar_comando_git(comando)

comando = 'git log'
ejecutar_comando_git(comando)

### RESET COMMITs ELIMINANDO LOS ARCHIVOS Y CAMBIOS HASTA UN COMMIT ESPECIFICO USANDO EL ID DEL COMMIT

In [ ]:
comando = 'git reset --hard 3b4edb5a785dcc5841b35cd2ad93033cd29cb730'
ejecutar_comando_git(comando)

## RESET HASTA UN COMMIT ESPECIFICO EN UN REPOSITORIO REMOTO <br>https://github.com/JorgeCardona/comandos_git</br>

In [ ]:
comando = 'git reset --hard d15fad27cb8d1bcb6a0e697ef02e2d787dc51b48'
ejecutar_comando_git(comando)

#### CONFIRMAR LOS CAMBIOS DEL RESET EN EL REPOSITORIO DE ORIGEN O REMOTO

In [ ]:
# confirmar los cambios al repositorio remoto el reset
comando = 'git push --force origin {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

#### LISTAR HISTORIAL DE CAMBIOS HECHOS EN EL REPOSITORIO LOCAL

In [ ]:
comando = 'git reflog'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

#### ELIMINAR LA RAMA EN EL REPOSITORIO DE ORIGEN O REMOTO

In [ ]:
comando = 'git checkout main'
ejecutar_comando_git(comando)

comando = 'git push origin -d {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

comando = 'git branch -D {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)

# <center> REVERTIR COMMITs DE UN REPOSITORIO</center>
![alternatvie text](revert.gif)

In [ ]:
nombre_rama = 'revert_branch'
generar_archivos_y_hacer_commit(nombre_rama=nombre_rama)

## REVERT HASTA UN COMMIT ESPECIFICO O UN HEAD~ EN UN REPOSITORIO REMOTO <br>https://github.com/JorgeCardona/comandos_git</br>

In [ ]:
# elimina todos los archivos o cambios hechos despues de este commit YA NO SE PUEDEN RECUPERAR ESOS ARCHIVOS
comando = 'git revert HEAD~12'
ejecutar_comando_git(comando)

comando = 'git log --oneline'
ejecutar_comando_git(comando)

#### CONFIRMAR LOS CAMBIOS DEL REVERT EN EL REPOSITORIO REMOTO

In [ ]:
# confirmar al repositorio remoto el reset
comando = 'git push --force origin {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

#### ELIMINAR LA RAMA EN EL REPOSITORIO DE ORIGEN O REMOTO

In [ ]:
comando = 'git checkout main'
ejecutar_comando_git(comando)

comando = 'git push origin -d {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

comando = 'git branch -D {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)

# <center> REBASE *JUNTAR VARIOS COMMITS EN UNO SOLO <br>REESCRIBIENDO EL HISTORIAL* COMMITs DE UN REPOSITORIO</br> <br>https://github.com/JorgeCardona/comandos_git</br></center>
![alternatvie text](rebase.gif)

In [ ]:
nombre_rama = 'rebase_branch'
generar_archivos_y_hacer_commit(nombre_rama=nombre_rama)

# EN LA CONSOLA, TERMINAL O UN IDE DE DESARROLLO
![alternatvie text](rebase_interactivo.gif)
![alternatvie text](rebase_tipos.png)
![alternatvie text](rebase_squash.gif)

##### SE ABRE AUTOMATICAMENTE EL EDITOR DE TEXTO, reemplazar la palabra **pick** por **squash** GUARDAR y CERRAR
##### SE ABRE AUTOMATICAMENTE EL EDITOR DE TEXTO, reemplazar los comentarios por el comentario que se quiere
##### subir los cambios al repositorio
##### git push --force origin rebase_branch

#### ELIMINAR LA RAMA DEL REPOSITORIO LOCAL Y REMOTO

In [ ]:
comando = 'git checkout main'
ejecutar_comando_git(comando)

comando = 'git push origin -d {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

comando = 'git branch -D {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)

# <center> MERGE RAMAS DE UN REPOSITORIO <br>https://github.com/JorgeCardona/comandos_git</br></center>
![alternatvie text](merge.gif)

In [ ]:
nombre_ramas = ['branch_uno','branch_dos']

for rama in nombre_ramas:
    generar_archivos_y_hacer_commit(rama, todos_los_archivos = False)

# <center> UNIR DOS RAMAS LOCALES </center>
![alternatvie text](merge_conflictos.gif)

# EN LA CONSOLA, TERMINAL O UN IDE DE DESARROLLO
# git merge branch_dos branch_uno

# Accept Current Change | Accept Incoming Change | Accept Both Changes | Compare Change
# <<<<<<< HEAD
# 1679198052.206751 (Current Change)
# =======
# 1679198046.401986 
# >>>>>>> branch_uno (Incoming Change)

# <center> UNIR CAMBIOS DE LA RAMA REMOTA A LA RAMA LOCAL </center> 

# EN LA CONSOLA DE VSCODE O UN IDE DE DESARROLLO
![alternatvie text](pull.gif)
# git pull origin branch_dos

# <center> UNIR CAMBIOS DE LA RAMA LOCAL A LA RAMA REMOTA </center> 

# EN LA CONSOLA DE VSCODE O UN IDE DE DESARROLLO
### hacer un git commit y despues un git pull

# Accept Current Change | Accept Incoming Change | Accept Both Changes | Compare Change
# <<<<<<< HEAD
# 1679198052.206751 (Current Change)
# =======
# 1679198046.401986 
# >>>>>>> branch_uno (Incoming Change)

#### ELIMINAR LA RAMA EN EL REPOSITORIO DE ORIGEN O REMOTO

In [ ]:
comando = 'git checkout main'
ejecutar_comando_git(comando)

for nombre_rama in nombre_ramas:
    comando = 'git push origin -d {nombre_rama}'.format(nombre_rama=nombre_rama)
    ejecutar_comando_git(comando)

    comando = 'git branch -D {nombre_rama}'.format(nombre_rama=nombre_rama)
    ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)

# <center> CHERRY PICK <br>https://github.com/JorgeCardona/comandos_git</br></center>
![alternatvie text](cherry-pick.gif)

In [ ]:
nombre_rama_inicial = 'cherry_pick_branch'
nombre_rama_final = 'hotfix_branch'

nombre_ramas = [nombre_rama_inicial,nombre_rama_final]

nombre_rama = nombre_rama_inicial
generar_archivos_y_hacer_commit(nombre_rama=nombre_rama)

nombre_rama = nombre_rama_final
generar_archivos_y_hacer_commit(nombre_rama=nombre_rama, todos_los_archivos=False)

## VER EL LOG DE LA RAMA A USAR COMO BASE PARA EL CHERRY_PICK

In [ ]:
comando = 'git log --oneline hotfix_branch'
ejecutar_comando_git(comando)


comando = 'git log --oneline cherry_pick_branch'
ejecutar_comando_git(comando)
ejecutar_comando_shell()

## CHERRY PICK A UN COMMIT DE LA RAMA **CHERRY_PICK** PARA UNIRLA CON LA RAMA ACTUAL **HOTFIX**

In [ ]:
comando = 'git cherry-pick 9ac61f3' # archivo main.py
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

In [ ]:
comando = 'git push origin'
ejecutar_comando_git(comando)

comando = 'git status'
ejecutar_comando_git(comando)

#### ELIMINAR LA RAMA EN EL REPOSITORIO DE ORIGEN O REMOTO

In [ ]:
comando = 'git checkout main'
ejecutar_comando_git(comando)

for nombre_rama in nombre_ramas:
    comando = 'git push origin -d {nombre_rama}'.format(nombre_rama=nombre_rama)
    ejecutar_comando_git(comando)

    comando = 'git branch -D {nombre_rama}'.format(nombre_rama=nombre_rama)
    ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)

# <center> TAGs <br>https://github.com/JorgeCardona/comandos_git</br></center>

In [ ]:
nombre_rama = 'tag_branch'
generar_archivos_y_hacer_commit(nombre_rama=nombre_rama)

### CREAR UN TAG

In [ ]:
nombre_tag = 'v0.0.0'
comando = 'git tag  {nombre_tag} -m "versión Inicial"'.format(nombre_tag=nombre_tag)
ejecutar_comando_git(comando)

comando = 'git tag'
ejecutar_comando_git(comando)

### LISTAR LOS TAGs LOCALES

In [ ]:
comando = 'git tag'
ejecutar_comando_git(comando)

### ELIMINAR UN TAG LOCAL

In [ ]:
comando = 'git tag -d {nombre_tag}'.format(nombre_tag=nombre_tag)
ejecutar_comando_git(comando)

comando = 'git tag'
ejecutar_comando_git(comando)

### CREAR MULTIPLES TAGs

In [ ]:
lista_ramas_locales = obtener_lista_nombres_ramas_git()

for indice, rama in enumerate(lista_ramas_locales):
 
    #cambia de rama para tener el contenido especifico de esa rama en el TAG
    comando = 'git checkout {rama}"'.format(rama=rama)
    ejecutar_comando_git(comando)
    
    # genera el nombre que tendra el TAG
    nombre_tag = 'v0.0.{indice}_{rama}'.format(rama=rama, indice=indice)
    comando = 'git tag {nombre_tag} -m "version {indice} del TAG"'.format(nombre_tag=nombre_tag, indice=indice)
    ejecutar_comando_git(comando)

comando = 'git tag'
ejecutar_comando_git(comando)

### LISTAR LOS TAGs Y VER SU DESCRIPCION

In [ ]:
comando = 'git tag -n'
ejecutar_comando_git(comando)

### PUBLICAR UN TAG ESPECIFICO A UN REPOSITORIO REMOTO <br>https://github.com/JorgeCardona/comandos_git</br>

In [ ]:
for indice, rama in enumerate([lista_ramas_locales[0]]):
    
    nombre_tag = 'v0.0.{indice}_{rama}'.format(rama=rama, indice=indice)
    comando = comando = 'git push origin {nombre_tag}'.format(nombre_tag=nombre_tag)
    ejecutar_comando_git(comando)

### LISTAR LOS TAGs REMOTOS

In [ ]:
comando = 'git ls-remote --tags'
ejecutar_comando_git(comando)

### PUBLICAR TODOS LOS TAGs NO PUBLICADOS EN UN SOLO PUSH AL RESPOSITORIO REMOTO

In [ ]:
comando = 'git push --tags'
ejecutar_comando_git(comando)

comando = 'git ls-remote --tags'
ejecutar_comando_git(comando)

### ELIMINAR LOS TAGs LOCALES Y REMOTOS

In [ ]:
for indice, rama in enumerate(lista_ramas_locales):
    
    nombre_tag = 'v0.0.{indice}_{rama}'.format(rama=rama, indice=indice)

    comando = 'git tag -d {nombre_tag}'.format(nombre_tag=nombre_tag)
    ejecutar_comando_git(comando)

    comando = 'git push --delete origin {nombre_tag}'.format(nombre_tag=nombre_tag)
    ejecutar_comando_git(comando)
    
comando = 'git tag'
ejecutar_comando_git(comando)

comando = 'git ls-remote --tags'
ejecutar_comando_git(comando)

#### ELIMINAR LA RAMA DEL REPOSITORIO LOCAL Y REMOTO


In [ ]:
comando = 'git checkout main'
ejecutar_comando_git(comando)

comando = 'git push origin -d {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

comando = 'git branch -D {nombre_rama}'.format(nombre_rama=nombre_rama)
ejecutar_comando_git(comando)

comando = 'git branch -a'
ejecutar_comando_git(comando)